In [ ]:
# # Clone the repository
# !git clone https://github.com/H4mzaCode/CompRecommendation.git
# %cd CompRecommendation

# # Import necessary libraries
# import json
# import pandas as pd

# # Load the dataset
# with open('appraisals_dataset.json', 'r') as f:
#     data = json.load(f)

# # Convert to DataFrame for easier analysis
# df = pd.DataFrame(data)
# # df.head()


# # Extract nested columns from the 'appraisals' column
# df['subject'] = df['appraisals'].apply(lambda x: x.get('subject'))
# df['comps'] = df['appraisals'].apply(lambda x: x.get('comps'))
# df['properties'] = df['appraisals'].apply(lambda x: x.get('properties'))

# # Drop the original 'appraisals' column
# df.drop(columns=['appraisals'], inplace=True)

# # Display the first few rows to verify the structure
# df.head()

In [ ]:
# Clone the repository
!git clone https://github.com/H4mzaCode/CompRecommendation.git
%cd CompRecommendation

Cloning into 'CompRecommendation'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 4.67 MiB | 4.40 MiB/s, done.
/content/CompRecommendation


## gla

In [ ]:
# Import necessary libraries
import json
import pandas as pd
import re
import numpy as np

# Function to standardize GLA values
def standardize_gla(gla_value):
    """
    Converts GLA values from various formats to integer or None

    Args:
        gla_value: The GLA value which could be an integer, string, or None

    Returns:
        int or None: Standardized GLA value
    """
    # If already None or np.nan, return None
    if gla_value is None or pd.isna(gla_value):
        return None

    # If already an integer, return as is
    if isinstance(gla_value, (int, float)) and not pd.isna(gla_value):
        return int(gla_value)

    # If it's a string, extract the numeric part
    if isinstance(gla_value, str):
        # Remove common text patterns and characters
        clean_str = gla_value.replace("SqFt", "").replace("sqft", "").replace("sq ft", "")
        clean_str = clean_str.replace("+/-", "").replace("~", "").strip()

        # Extract the first number found (handles cases like "2100+/-SqFt")
        match = re.search(r'(\d+)', clean_str)
        if match:
            return int(match.group(1))

    # If we couldn't extract a value, return None
    return None

# Load the dataset with original structure
with open('appraisals_dataset.json', 'r') as f:
    data = json.load(f)

# Process GLA values while maintaining the original structure
print("Standardizing GLA values...")

# Process all appraisals
for i, appraisal in enumerate(data['appraisals']):
    # Process subject property
    if 'subject' in appraisal and appraisal['subject'] and 'gla' in appraisal['subject']:
        original = appraisal['subject']['gla']
        standardized = standardize_gla(original)
        data['appraisals'][i]['subject']['gla'] = standardized

        # Print sample conversions (first 3 only)
        if i < 3:
            print(f"  Subject in appraisal {i}: {original} → {standardized}")

    # Process comps
    if 'comps' in appraisal and appraisal['comps']:
        for j, comp in enumerate(appraisal['comps']):
            if comp and 'gla' in comp:
                original = comp['gla']
                standardized = standardize_gla(original)
                data['appraisals'][i]['comps'][j]['gla'] = standardized

                # Print sample conversions (very limited to avoid clutter)
                if i < 2 and j < 2:
                    print(f"  Comp {j} in appraisal {i}: {original} → {standardized}")

    # Process properties
    if 'properties' in appraisal and appraisal['properties']:
        for j, prop in enumerate(appraisal['properties']):
            if prop and 'gla' in prop:
                original = prop['gla']
                standardized = standardize_gla(original)
                data['appraisals'][i]['properties'][j]['gla'] = standardized

                # Print sample conversions (very limited to avoid clutter)
                if i < 2 and j < 2:
                    print(f"  Property {j} in appraisal {i}: {original} → {standardized}")

# Verify a few examples to confirm standardization worked
print("\nVerification of standardized data:")
if len(data['appraisals']) > 0:
    # Verify subject
    if 'subject' in data['appraisals'][0] and 'gla' in data['appraisals'][0]['subject']:
        print(f"  Subject GLA (first appraisal): {data['appraisals'][0]['subject']['gla']}")

    # Verify comp
    if ('comps' in data['appraisals'][0] and data['appraisals'][0]['comps'] and
        len(data['appraisals'][0]['comps']) > 0 and 'gla' in data['appraisals'][0]['comps'][0]):
        print(f"  First comp GLA (first appraisal): {data['appraisals'][0]['comps'][0]['gla']}")

    # Verify property
    if ('properties' in data['appraisals'][0] and data['appraisals'][0]['properties'] and
        len(data['appraisals'][0]['properties']) > 0 and 'gla' in data['appraisals'][0]['properties'][0]):
        print(f"  First property GLA (first appraisal): {data['appraisals'][0]['properties'][0]['gla']}")

# Save the standardized data with the original structure
with open('standardized_appraisals_dataset.json', 'w') as f:
    json.dump(data, f, indent=2)

print("\nStandardization complete! File saved as 'standardized_appraisals_dataset.json' with the original structure.")

Standardizing GLA values...
  Subject in appraisal 0: 1044 → 1044
  Comp 0 in appraisal 0: 1044 SqFt → 1044
  Comp 1 in appraisal 0: 1044 SqFt → 1044
  Property 0 in appraisal 0: 1500 → 1500
  Property 1 in appraisal 0: 1750 → 1750
  Subject in appraisal 1: 1500 SqFt → 1500
  Comp 0 in appraisal 1: 1602+/-SqFt → 1602
  Comp 1 in appraisal 1: 2100+/-SqFt → 2100
  Property 0 in appraisal 1: 1136 → 1136
  Property 1 in appraisal 1: 1026 → 1026
  Subject in appraisal 2: 3000 SqFt → 3000

Verification of standardized data:
  Subject GLA (first appraisal): 1044
  First comp GLA (first appraisal): 1044
  First property GLA (first appraisal): 1500

Standardization complete! File saved as 'standardized_appraisals_dataset.json' with the original structure.


## Address

In [ ]:
# Import necessary libraries
import json
import re

# Function to standardize addresses
def standardize_address(address_data):
    """
    Unifies address formats from different structures (subject, comps, properties) into a consistent format.

    For example:
    - Complete address: "7180 207 HWY Halifax NS B0J2L0"
    - Split address: address="64 Deermist Dr", city_province="Porters Lake NS B3E 1P3"
    - Multiple fields: address="638 Myers Point Road", city="Nova Scotia", province="Nova Scotia", postal_code="B0J 1P0"

    Returns a standardized address string.
    """
    # Case 1: If it's a dictionary containing address components
    if isinstance(address_data, dict):
        address_parts = []

        # Add the street address if available
        if 'address' in address_data and address_data['address']:
            address_parts.append(address_data['address'].strip())

        # Handle comps format where city and province are combined
        if 'city_province' in address_data and address_data['city_province']:
            address_parts.append(address_data['city_province'].strip())

        # Handle properties format with separate city, province, postal_code
        else:
            location_parts = []

            # Add city if available and not redundant
            if 'city' in address_data and address_data['city'] and address_data['city'].strip() not in ["", "Nova Scotia", "Ontario"]:
                location_parts.append(address_data['city'].strip())

            # Add province if available
            if 'province' in address_data and address_data['province']:
                # Convert full province name to abbreviation
                province = address_data['province'].strip()
                if province == "Nova Scotia":
                    location_parts.append("NS")
                elif province == "Ontario":
                    location_parts.append("ON")
                else:
                    location_parts.append(province)

            # Add postal code if available
            if 'postal_code' in address_data and address_data['postal_code']:
                location_parts.append(address_data['postal_code'].strip())

            # Join the location parts and add to address parts
            if location_parts:
                address_parts.append(" ".join(location_parts))

        # Join all address parts into a single string
        return " ".join(address_parts)

    # Case 2: It's already a string address
    elif isinstance(address_data, str):
        return address_data.strip()

    # Default case
    return ""

# Function to combine address components in comps
def process_comps(comps):
    """Process and standardize addresses in comps section"""
    processed_comps = []

    for comp in comps:
        if comp is None:
            processed_comps.append(comp)
            continue

        processed_comp = comp.copy()

        # Extract address and city_province
        address = comp.get('address', '')
        city_province = comp.get('city_province', '')

        # If both fields exist, combine them
        if address and city_province:
            processed_comp['address'] = f"{address.strip()} {city_province.strip()}"

            # Remove the now redundant city_province field
            if 'city_province' in processed_comp:
                del processed_comp['city_province']

        processed_comps.append(processed_comp)

    return processed_comps

# Function to combine address components in properties
def process_properties(properties):
    """Process and standardize addresses in properties section"""
    processed_properties = []

    for prop in properties:
        if prop is None:
            processed_properties.append(prop)
            continue

        processed_prop = prop.copy()

        # Create full address from components
        address_components = {
            'address': prop.get('address', ''),
            'city': prop.get('city', ''),
            'province': prop.get('province', ''),
            'postal_code': prop.get('postal_code', '')
        }

        # Create standardized address
        full_address = standardize_address(address_components)
        if full_address:
            processed_prop['address'] = full_address

            # Remove the now redundant fields
            if 'city' in processed_prop:
                del processed_prop['city']
            if 'province' in processed_prop:
                del processed_prop['province']
            if 'postal_code' in processed_prop:
                del processed_prop['postal_code']

        processed_properties.append(processed_prop)

    return processed_properties

# Load the standardized dataset
print("Loading standardized_appraisals_dataset.json...")
try:
    with open('standardized_appraisals_dataset.json', 'r') as f:
        data = json.load(f)
    print("File loaded successfully.")
except FileNotFoundError:
    print("Error: standardized_appraisals_dataset.json not found.")
    exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in standardized_appraisals_dataset.json.")
    exit(1)

# Process and standardize addresses throughout the dataset
print("Standardizing addresses...")
address_count = 0

for i, appraisal in enumerate(data['appraisals']):
    # Process subject
    if 'subject' in appraisal and appraisal['subject']:
        # Subject addresses are already in the desired format, but let's clean them up
        if 'address' in appraisal['subject'] and appraisal['subject']['address']:
            appraisal['subject']['address'] = appraisal['subject']['address'].strip()
            address_count += 1

    # Process comps
    if 'comps' in appraisal and appraisal['comps']:
        data['appraisals'][i]['comps'] = process_comps(appraisal['comps'])
        address_count += sum(1 for comp in appraisal['comps'] if comp and 'address' in comp)

    # Process properties
    if 'properties' in appraisal and appraisal['properties']:
        data['appraisals'][i]['properties'] = process_properties(appraisal['properties'])
        address_count += sum(1 for prop in appraisal['properties'] if prop and 'address' in prop)

print(f"Standardized {address_count} addresses.")

# Print verification examples
print("\nVerification examples:")
example_types = ['subject', 'comps', 'properties']
count = 0

for appraisal in data['appraisals']:
    for example_type in example_types:
        if example_type in appraisal:
            if example_type == 'subject' and 'address' in appraisal[example_type]:
                print(f"Subject address: {appraisal[example_type]['address']}")
                count += 1
            elif example_type in ['comps', 'properties'] and appraisal[example_type]:
                for item in appraisal[example_type]:
                    if item and 'address' in item:
                        item_type = "Comp" if example_type == 'comps' else "Property"
                        print(f"{item_type} address: {item['address']}")
                        count += 1
                        break

        if count >= 5:  # Limit the number of examples to display
            break

    if count >= 5:
        break

# Save the updated data
print("\nSaving updated data...")
with open('unified_addresses_appraisals_dataset.json', 'w') as f:
    json.dump(data, f, indent=2)

print("\nAddress standardization complete! File saved as 'unified_addresses_appraisals_dataset.json'.")
print("You can now download this file from your Google Colab environment.")

Loading standardized_appraisals_dataset.json...
File loaded successfully.
Standardizing addresses...
Standardized 10172 addresses.

Verification examples:
Subject address: 142-950 Oakview Ave Kingston ON K7M 6W8
Comp address: 930 Amberdale Cres Kingston ON K7M 6V1
Property address: 463 Conservatory Dr Kingston ON K7M 9C8
Subject address: 7180 207 HWY Halifax NS B0J2L0
Comp address: 64 Deermist Dr Porters Lake NS B3E 1P3

Saving updated data...

Address standardization complete! File saved as 'unified_addresses_appraisals_dataset.json'.
You can now download this file from your Google Colab environment.


## Step 1: Standardize Dates

In [4]:
# Import necessary libraries
import json
import re
import datetime
import pandas as pd

# Function to standardize dates to YYYY-MM-DD format
def standardize_date(date_value):
    """
    Converts date values from various formats to YYYY-MM-DD format

    Args:
        date_value: The date value which could be in different formats:
                    - "Apr/11/2025"
                    - "Oct/25/2024"
                    - "2025-01-13"
                    - or other formats

    Returns:
        str or None: Standardized date in YYYY-MM-DD format, or None if conversion failed
    """
    # If already None or empty, return None
    if date_value is None or date_value == "":
        return None

    # If already in YYYY-MM-DD format, return as is
    if isinstance(date_value, str) and re.match(r'^\d{4}-\d{2}-\d{2}$', date_value):
        return date_value

    # Try to parse with various formats
    if isinstance(date_value, str):
        # Format: "Apr/11/2025" or "Oct/25/2024"
        month_pattern = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)/(\d{1,2})/(\d{4})'
        month_match = re.match(month_pattern, date_value)
        if month_match:
            month_map = {
                'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
            }
            month_name, day, year = month_match.groups()
            month = month_map[month_name]
            try:
                date_obj = datetime.datetime(int(year), month, int(day))
                return date_obj.strftime('%Y-%m-%d')
            except ValueError:
                pass

        # Try pandas to_datetime which can handle many formats
        try:
            date_obj = pd.to_datetime(date_value)
            return date_obj.strftime('%Y-%m-%d')
        except:
            pass

    # If we couldn't parse the date, return the original value
    return date_value

# Load the standardized dataset
print("Loading unified_addresses_appraisals_dataset.json...")
try:
    with open('unified_addresses_appraisals_dataset.json', 'r') as f:
        data = json.load(f)
    print("File loaded successfully.")
except FileNotFoundError:
    print("Error: unified_addresses_appraisals_dataset.json not found.")
    exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in unified_addresses_appraisals_dataset.json.")
    exit(1)

# Process all date fields throughout the dataset
print("Standardizing date fields...")
date_count = 0

for i, appraisal in enumerate(data['appraisals']):
    # Process subject
    if 'subject' in appraisal and appraisal['subject']:
        subject = appraisal['subject']
        if 'effective_date' in subject and subject['effective_date']:
            original = subject['effective_date']
            standardized = standardize_date(original)
            data['appraisals'][i]['subject']['effective_date'] = standardized
            date_count += 1

            # Print sample conversions (first few only)
            if i < 3:
                print(f"  Subject effective_date in appraisal {i}: {original} → {standardized}")

    # Process comps
    if 'comps' in appraisal and appraisal['comps']:
        for j, comp in enumerate(appraisal['comps']):
            if comp and 'sale_date' in comp:
                original = comp['sale_date']
                standardized = standardize_date(original)
                data['appraisals'][i]['comps'][j]['sale_date'] = standardized
                date_count += 1

                # Print sample conversions (limited to avoid clutter)
                if i < 2 and j < 2:
                    print(f"  Comp {j} sale_date in appraisal {i}: {original} → {standardized}")

    # Process properties
    if 'properties' in appraisal and appraisal['properties']:
        for j, prop in enumerate(appraisal['properties']):
            if prop and 'close_date' in prop:
                original = prop['close_date']
                standardized = standardize_date(original)
                data['appraisals'][i]['properties'][j]['close_date'] = standardized
                date_count += 1

                # Print sample conversions (limited to avoid clutter)
                if i < 2 and j < 2:
                    print(f"  Property {j} close_date in appraisal {i}: {original} → {standardized}")

print(f"Standardized {date_count} date fields.")

# Verify date standardization
print("\nVerification of standardized dates:")
example_types = {'subject': 'effective_date', 'comps': 'sale_date', 'properties': 'close_date'}
count = 0

for appraisal in data['appraisals']:
    for section, date_field in example_types.items():
        if section in appraisal:
            if section == 'subject' and date_field in appraisal[section]:
                print(f"Subject {date_field}: {appraisal[section][date_field]}")
                count += 1
            elif section in ['comps', 'properties'] and appraisal[section]:
                for item in appraisal[section]:
                    if item and date_field in item:
                        section_name = "Comp" if section == 'comps' else "Property"
                        print(f"{section_name} {date_field}: {item[date_field]}")
                        count += 1
                        break

        if count >= 6:  # Limit the number of examples to display
            break

    if count >= 6:
        break

# Save the updated data
print("\nSaving updated data...")
with open('date_standardized_appraisals_dataset.json', 'w') as f:
    json.dump(data, f, indent=2)

print("\nDate standardization complete! File saved as 'date_standardized_appraisals_dataset.json'.")
print("You can now download this file from your Google Colab environment.")

Loading unified_addresses_appraisals_dataset.json...
File loaded successfully.
Standardizing date fields...
  Subject effective_date in appraisal 0: Apr/11/2025 → 2025-04-11
  Comp 0 sale_date in appraisal 0: Oct/25/2024 → 2024-10-25
  Comp 1 sale_date in appraisal 0: Feb/05/2025 → 2025-02-05
  Property 0 close_date in appraisal 0: 2025-01-13 → 2025-01-13
  Property 1 close_date in appraisal 0: 2025-01-13 → 2025-01-13
  Subject effective_date in appraisal 1: Apr/17/2025 → 2025-04-17
  Comp 0 sale_date in appraisal 1: Jan/16/2025 → 2025-01-16
  Comp 1 sale_date in appraisal 1: Dec/21/2024 → 2024-12-21
  Property 0 close_date in appraisal 1: 2024-10-30 → 2024-10-30
  Property 1 close_date in appraisal 1: 2024-11-01 → 2024-11-01
  Subject effective_date in appraisal 2: May/01/2025 → 2025-05-01
Standardized 10172 date fields.

Verification of standardized dates:
Subject effective_date: 2025-04-11
Comp sale_date: 2024-10-25
Property close_date: 2025-01-13
Subject effective_date: 2025-04-17


In [5]:
# Import necessary libraries
import json
import re
import datetime
import pandas as pd

# Function to standardize dates to YYYY-MM-DD format
def standardize_date(date_value):
    """
    Converts date values from various formats to YYYY-MM-DD format

    Args:
        date_value: The date value which could be in different formats:
                    - "Apr/11/2025"
                    - "Oct/25/2024"
                    - "2025-01-13"
                    - or other formats

    Returns:
        str or None: Standardized date in YYYY-MM-DD format, or None if conversion failed
    """
    # If already None or empty, return None
    if date_value is None or date_value == "":
        return None

    # If already in YYYY-MM-DD format, return as is
    if isinstance(date_value, str) and re.match(r'^\d{4}-\d{2}-\d{2}$', date_value):
        return date_value

    # Try to parse with various formats
    if isinstance(date_value, str):
        # Format: "Apr/11/2025" or "Oct/25/2024"
        month_pattern = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)/(\d{1,2})/(\d{4})'
        month_match = re.match(month_pattern, date_value)
        if month_match:
            month_map = {
                'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
            }
            month_name, day, year = month_match.groups()
            month = month_map[month_name]
            try:
                date_obj = datetime.datetime(int(year), month, int(day))
                return date_obj.strftime('%Y-%m-%d')
            except ValueError:
                pass

        # Try pandas to_datetime which can handle many formats
        try:
            date_obj = pd.to_datetime(date_value)
            return date_obj.strftime('%Y-%m-%d')
        except:
            pass

    # If we couldn't parse the date, return the original value
    return date_value

# Load the standardized dataset
print("Loading unified_addresses_appraisals_dataset.json...")
try:
    with open('unified_addresses_appraisals_dataset.json', 'r') as f:
        data = json.load(f)
    print("File loaded successfully.")
except FileNotFoundError:
    print("Error: unified_addresses_appraisals_dataset.json not found.")
    exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in unified_addresses_appraisals_dataset.json.")
    exit(1)

# Process all date fields throughout the dataset
print("Standardizing date fields...")
date_count = 0
field_renames = 0

for i, appraisal in enumerate(data['appraisals']):
    # Process subject
    if 'subject' in appraisal and appraisal['subject']:
        subject = appraisal['subject']
        if 'effective_date' in subject and subject['effective_date']:
            original = subject['effective_date']
            standardized = standardize_date(original)

            # Add standardized date with new field name
            data['appraisals'][i]['subject']['date'] = standardized

            # Remove the old field
            del data['appraisals'][i]['subject']['effective_date']

            date_count += 1
            field_renames += 1

            # Print sample conversions (first few only)
            if i < 3:
                print(f"  Subject effective_date → date in appraisal {i}: {original} → {standardized}")

    # Process comps
    if 'comps' in appraisal and appraisal['comps']:
        for j, comp in enumerate(appraisal['comps']):
            if comp and 'sale_date' in comp:
                original = comp['sale_date']
                standardized = standardize_date(original)

                # Add standardized date with new field name
                data['appraisals'][i]['comps'][j]['date'] = standardized

                # Remove the old field
                del data['appraisals'][i]['comps'][j]['sale_date']

                date_count += 1
                field_renames += 1

                # Print sample conversions (limited to avoid clutter)
                if i < 2 and j < 2:
                    print(f"  Comp {j} sale_date → date in appraisal {i}: {original} → {standardized}")

    # Process properties
    if 'properties' in appraisal and appraisal['properties']:
        for j, prop in enumerate(appraisal['properties']):
            if prop and 'close_date' in prop:
                original = prop['close_date']
                standardized = standardize_date(original)

                # Add standardized date with new field name
                data['appraisals'][i]['properties'][j]['date'] = standardized

                # Remove the old field
                del data['appraisals'][i]['properties'][j]['close_date']

                date_count += 1
                field_renames += 1

                # Print sample conversions (limited to avoid clutter)
                if i < 2 and j < 2:
                    print(f"  Property {j} close_date → date in appraisal {i}: {original} → {standardized}")

print(f"Standardized {date_count} date fields and renamed {field_renames} fields to 'date'.")

# Verify date standardization
print("\nVerification of standardized dates:")
count = 0

for appraisal in data['appraisals']:
    # Check subject
    if 'subject' in appraisal and appraisal['subject'] and 'date' in appraisal['subject']:
        print(f"Subject date: {appraisal['subject']['date']}")
        count += 1

    # Check comps
    if 'comps' in appraisal and appraisal['comps']:
        for comp in appraisal['comps']:
            if comp and 'date' in comp:
                print(f"Comp date: {comp['date']}")
                count += 1
                break  # Just show one example

    # Check properties
    if 'properties' in appraisal and appraisal['properties']:
        for prop in appraisal['properties']:
            if prop and 'date' in prop:
                print(f"Property date: {prop['date']}")
                count += 1
                break  # Just show one example

    if count >= 6:  # Limit the number of examples to display
        break

# Save the updated data
print("\nSaving updated data...")
with open('date_standardized_appraisals_dataset.json', 'w') as f:
    json.dump(data, f, indent=2)

print("\nDate standardization complete! File saved as 'date_standardized_appraisals_dataset.json'.")
print("You can now download this file from your Google Colab environment.")

Loading unified_addresses_appraisals_dataset.json...
File loaded successfully.
Standardizing date fields...
  Subject effective_date → date in appraisal 0: Apr/11/2025 → 2025-04-11
  Comp 0 sale_date → date in appraisal 0: Oct/25/2024 → 2024-10-25
  Comp 1 sale_date → date in appraisal 0: Feb/05/2025 → 2025-02-05
  Property 0 close_date → date in appraisal 0: 2025-01-13 → 2025-01-13
  Property 1 close_date → date in appraisal 0: 2025-01-13 → 2025-01-13
  Subject effective_date → date in appraisal 1: Apr/17/2025 → 2025-04-17
  Comp 0 sale_date → date in appraisal 1: Jan/16/2025 → 2025-01-16
  Comp 1 sale_date → date in appraisal 1: Dec/21/2024 → 2024-12-21
  Property 0 close_date → date in appraisal 1: 2024-10-30 → 2024-10-30
  Property 1 close_date → date in appraisal 1: 2024-11-01 → 2024-11-01
  Subject effective_date → date in appraisal 2: May/01/2025 → 2025-05-01
Standardized 10172 date fields and renamed 10172 fields to 'date'.

Verification of standardized dates:
Subject date: 202

## Deduplicate

In [6]:
# Import necessary libraries
import json
import re
from collections import defaultdict

# Load the date-standardized dataset
print("Loading date_standardized_appraisals_dataset.json...")
try:
    with open('date_standardized_appraisals_dataset.json', 'r') as f:
        data = json.load(f)
    print("File loaded successfully.")
except FileNotFoundError:
    print("Error: date_standardized_appraisals_dataset.json not found.")
    exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in date_standardized_appraisals_dataset.json.")
    exit(1)

# Function to normalize address for comparison
def normalize_address(address):
    """
    Normalizes an address by removing extra spaces, standardizing common abbreviations,
    and converting to lowercase for case-insensitive comparison.

    Args:
        address: The address string to normalize

    Returns:
        str: Normalized address string
    """
    if not address or not isinstance(address, str):
        return ""

    # Convert to lowercase
    address = address.lower()

    # Replace multiple spaces with a single space
    address = re.sub(r'\s+', ' ', address)

    # Standardize common abbreviations
    address = address.replace(" dr ", " drive ")
    address = address.replace(" dr,", " drive,")
    address = address.replace(" dr.", " drive.")
    address = re.sub(r'(\s|^)dr$', r'\1drive', address)

    address = address.replace(" st ", " street ")
    address = address.replace(" st,", " street,")
    address = address.replace(" st.", " street.")
    address = re.sub(r'(\s|^)st$', r'\1street', address)

    address = address.replace(" ave ", " avenue ")
    address = address.replace(" ave,", " avenue,")
    address = address.replace(" ave.", " avenue.")
    address = re.sub(r'(\s|^)ave$', r'\1avenue', address)

    address = address.replace(" cres ", " crescent ")
    address = address.replace(" cres,", " crescent,")
    address = address.replace(" cres.", " crescent.")
    address = re.sub(r'(\s|^)cres$', r'\1crescent', address)

    # Remove trailing whitespace
    address = address.strip()

    return address

# Track total number of deduplications
total_deduped = 0
all_dups_found = 0

# Process properties for each appraisal
print("\nChecking for duplicate properties...")
for i, appraisal in enumerate(data['appraisals']):
    if 'properties' not in appraisal or not appraisal['properties']:
        continue

    # Track unique properties by normalized address
    unique_properties = {}
    duplicates_found = 0

    # Group properties by normalized address
    address_groups = defaultdict(list)
    for prop in appraisal['properties']:
        if 'address' not in prop or not prop['address']:
            continue

        norm_address = normalize_address(prop['address'])
        address_groups[norm_address].append(prop)

    # Process groups with duplicates
    new_properties = []
    for norm_address, props in address_groups.items():
        if len(props) > 1:
            # Found duplicate(s)
            duplicates_found += len(props) - 1
            all_dups_found += len(props) - 1

            # Print first few duplicates for verification
            if i < 3 and duplicates_found <= 3:
                print(f"\nDuplicate found in appraisal {i} for address: {props[0]['address']}")
                for idx, p in enumerate(props):
                    print(f"  Duplicate {idx+1}: ID={p.get('id', 'N/A')}, "
                          f"GLA={p.get('gla', 'N/A')}, "
                          f"Price={p.get('close_price', 'N/A')}")

            # Merge duplicate properties by keeping the most complete one
            # (Choose the one with more non-null fields)
            best_prop = max(props, key=lambda p: sum(1 for v in p.values() if v is not None))
            new_properties.append(best_prop)
        else:
            # No duplicates, keep the property
            new_properties.append(props[0])

    # Replace the properties list with deduplicated one
    removed_count = len(appraisal['properties']) - len(new_properties)
    total_deduped += removed_count
    data['appraisals'][i]['properties'] = new_properties

    # Print summary for this appraisal
    if removed_count > 0:
        print(f"Appraisal {i}: Removed {removed_count} duplicate properties")

# Check for duplicates in comps
print("\nChecking for duplicate comps...")
comps_deduped = 0

for i, appraisal in enumerate(data['appraisals']):
    if 'comps' not in appraisal or not appraisal['comps']:
        continue

    # Group comps by normalized address
    address_groups = defaultdict(list)
    for comp in appraisal['comps']:
        if 'address' not in comp or not comp['address']:
            continue

        norm_address = normalize_address(comp['address'])
        address_groups[norm_address].append(comp)

    # Process groups with duplicates
    new_comps = []
    removed_count = 0

    for norm_address, comps in address_groups.items():
        if len(comps) > 1:
            # Found duplicate(s)
            removed_count += len(comps) - 1
            comps_deduped += len(comps) - 1
            all_dups_found += len(comps) - 1

            # Print first few duplicates for verification
            if i < 3 and removed_count <= 3:
                print(f"\nDuplicate found in appraisal {i} for comp address: {comps[0]['address']}")
                for idx, c in enumerate(comps):
                    print(f"  Duplicate {idx+1}: "
                          f"Sale Price={c.get('sale_price', 'N/A')}, "
                          f"Date={c.get('date', 'N/A')}")

            # Keep the most complete comp
            best_comp = max(comps, key=lambda c: sum(1 for v in c.values() if v is not None))
            new_comps.append(best_comp)
        else:
            # No duplicates, keep the comp
            new_comps.append(comps[0])

    # Replace the comps list with deduplicated one
    if removed_count > 0:
        data['appraisals'][i]['comps'] = new_comps
        print(f"Appraisal {i}: Removed {removed_count} duplicate comps")
        total_deduped += removed_count

# Print overall summary
print(f"\nDeduplication complete! Removed {total_deduped} duplicate entries in total.")
print(f"  - Found {all_dups_found} duplicate entries across all appraisals")
print(f"  - Properties deduplicated: {total_deduped - comps_deduped}")
print(f"  - Comps deduplicated: {comps_deduped}")

# Save the updated data
print("\nSaving deduplicated data...")
with open('deduplicated_appraisals_dataset.json', 'w') as f:
    json.dump(data, f, indent=2)

print("\nDeduplication complete! File saved as 'deduplicated_appraisals_dataset.json'.")

Loading date_standardized_appraisals_dataset.json...
File loaded successfully.

Checking for duplicate properties...

Duplicate found in appraisal 0 for address: 463 Conservatory Dr Kingston ON K7M 9C8
  Duplicate 1: ID=367, GLA=1500, Price=674000.0
  Duplicate 2: ID=163443, GLA=1750, Price=674000.0

Duplicate found in appraisal 0 for address: 311 Janette St Kingston ON K7P 0K8
  Duplicate 1: ID=378, GLA=1500, Price=585000.0
  Duplicate 2: ID=130023, GLA=1300, Price=585000.0

Duplicate found in appraisal 0 for address: 786 HIGH GATE PARK Dr Kingston ON K7M 5Z8
  Duplicate 1: ID=2783, GLA=1300, Price=495000.0
  Duplicate 2: ID=138739, GLA=1300, Price=495000.0
Appraisal 0: Removed 27 duplicate properties

Duplicate found in appraisal 2 for address: 38 Mistywood Street Kitchener ON N2P 2N1
  Duplicate 1: ID=191977, GLA=2441, Price=1100000.0
  Duplicate 2: ID=285076, GLA=2441, Price=1100000.0

Duplicate found in appraisal 2 for address: Unit 20A - 388 Old Huron Road Kitchener ON N2R 0J6
  